In [1]:
import warnings 
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np

from tqdm.auto import tqdm
from sklearn.metrics.pairwise import euclidean_distances

pd.options.display.max_columns=100

In [3]:
data_path = "data/"
orders = pd.read_csv(data_path +"orders.csv")
vendors = pd.read_csv(data_path +"vendors.csv")
train_locations = pd.read_csv(data_path +"train_locations.csv")
test_locations = pd.read_csv(data_path +"test_locations.csv")
train_customers = pd.read_csv(data_path +"train_customers.csv")
test_customers = pd.read_csv(data_path +"train_customers.csv")
SampleSubmission = pd.read_csv(data_path +"SampleSubmission.csv")

In [4]:
train_locations.head(2)

,customer_id,location_number,location_type,latitude,longitude
0,02SFNJH,0,NaN,1.682392,-78.789737
1,02SFNJH,1,NaN,1.679137,0.766823


In [5]:
orders.head(2)

,akeed_order_id,customer_id,item_count,grand_total,payment_mode,promo_code,vendor_discount_amount,promo_code_discount_percentage,is_favorite,is_rated,vendor_rating,driver_rating,deliverydistance,preparationtime,delivery_time,order_accepted_time,driver_accepted_time,ready_for_pickup_time,picked_up_time,delivered_time,delivery_date,vendor_id,created_at,LOCATION_NUMBER,LOCATION_TYPE,CID X LOC_NUM X VENDOR
0,163238.0,92PEE24,1.0,7.6,2,NaN,0.0,NaN,NaN,No,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-07-31 05:30:00,105,2019-08-01 05:30:16,0,NaN,92PEE24 X 0 X 105
1,163240.0,QS68UD8,1.0,8.7,1,NaN,0.0,NaN,NaN,No,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-07-31 05:30:00,294,2019-08-01 05:31:10,0,Work,QS68UD8 X 0 X 294


In [7]:
train_merge = train_locations.merge(orders, how="inner", on="customer_id")
train_merge.head(2)

,customer_id,location_number,location_type,latitude,longitude,akeed_order_id,item_count,grand_total,payment_mode,promo_code,vendor_discount_amount,promo_code_discount_percentage,is_favorite,is_rated,vendor_rating,driver_rating,deliverydistance,preparationtime,delivery_time,order_accepted_time,driver_accepted_time,ready_for_pickup_time,picked_up_time,delivered_time,delivery_date,vendor_id,created_at,LOCATION_NUMBER,LOCATION_TYPE,CID X LOC_NUM X VENDOR
0,02SFNJH,0,NaN,1.682392,-78.789737,235719.0,2.0,17.1,2,NaN,0.0,NaN,No,No,0.0,0.0,0.0,NaN,NaN,2019-10-25 14:11:24,NaN,2019-10-25 14:12:58,2019-10-25 18:29:40,2019-10-25 18:56:53,NaN,199,2019-10-25 18:09:38,1,Other,02SFNJH X 1 X 199
1,02SFNJH,0,NaN,1.682392,-78.789737,253305.0,6.0,26.4,1,NaN,0.0,NaN,No,No,0.0,0.0,5.1,40.0,NaN,2019-11-07 18:32:29,NaN,2019-11-07 18:46:20,2019-11-07 22:48:21,2019-11-07 23:13:14,NaN,159,2019-11-07 22:32:15,2,NaN,02SFNJH X 2 X 159


In [8]:
train_good_merge = train_merge[train_merge["location_number"] == train_merge["LOCATION_NUMBER"]]

In [12]:
train_good_merge = train_good_merge.fillna(0)
test_locations = test_locations.fillna(0)

In [16]:
test_locations.head(2)

,customer_id,location_number,location_type,latitude,longitude
0,Z59FTQD,0,0,126.032278,-9.106019
1,0JP29SK,0,Home,0.278709,-78.623847


In [17]:
len(test_locations)

16720

In [19]:
test_loc = test_locations[["latitude", "longitude"]].values
train_loc = train_good_merge[["latitude", "longitude"]].values
train_vend = train_good_merge["vendor_id"].values

In [24]:
euc = euclidean_distances(test_loc, train_loc)

In [26]:
vendors_for_users = []
for i in tqdm(range(len(test_loc))):
    unique, counts = np.unique(train_vend[np.argsort(euc[i])[:25]], return_counts=True)
    vendors_for_users.append(unique[counts>=2])

In [28]:
vendors_id = set(list(vendors["id"].values))

In [29]:
final_list = []
for i, user_loc_val  in tqdm(enumerate(test_locations[["customer_id", "location_number"]].values)):
    vendors_to_add = vendors_for_users[i]
    vendors_not_to_add = list(vendors_id - set(vendors_to_add))
    
    for vend in vendors_to_add:
        final_list.append([str(user_loc_val[0])+" X "+str(int(user_loc_val[1]))+" X "+str(int(vend)), 1])
    for vend in vendors_not_to_add:
        final_list.append([str(user_loc_val[0])+" X "+str(int(user_loc_val[1]))+" X "+str(int(vend)), 0])

In [30]:
submit = pd.DataFrame(final_list, columns = ["CID X LOC_NUM X VENDOR", "target"])

In [31]:
submit["target"].value_counts()

0    1589489
1      82511
Name: target, dtype: int64

In [32]:
submit.to_csv("knn_25_2.csv", index=None)